<div class="alert alert-block alert-info" >
    <h1>Robot Perception</h1>
    <h3>General Information:</h3>
    <p>Please do not add or delete any cells. Answers belong into the corresponding cells (below the question). If a function is given (either as a signature or a full function), you should not change the name, arguments or return value of the function.<br><br> If you encounter empty cells underneath the answer that can not be edited, please ignore them, they are for testing purposes.<br><br>When editing an assignment there can be the case that there are variables in the kernel. To make sure your assignment works, please restart the kernel and run all cells before submitting (e.g. via <i>Kernel -> Restart & Run All</i>).</p>
    <p>Code cells where you are supposed to give your answer often include the line  ```raise NotImplementedError```. This makes it easier to automatically grade answers. If you edit the cell please outcomment or delete this line.</p>
    <br>The server resource is limited to 2 core cpu and 1GB RAM at max per user. If you use more than that, the kernel may die. Nevertheless, you can bring it up again by restaring the kernel (Kernel -> Restart and clear output).<br>
    <h3>Submission:</h3>
    <p>Please submit your notebook via the web interface (in the main view -> Assignments -> Submit). The assignments are due on <b>Monday at 0:00.</b> (i.e. Sunday 23:59 + 1 min)</p>
    <h3>Group Work:</h3>
    <p>You are allowed to work in groups of up to two people. Please enter the UID (your username here) of each member of the group into the next cell. We apply plagiarism checking, so do not submit solutions from other people except your team members. If an assignment has a copied solution, the task will be graded with 0 points for all people with the same solution.</p>
    <p><b>YOU SHOULD ONLY SUBMIT EXACTLY ONE PER GROUP</b></p>
    <h3>Questions about the Assignment:</h3>
    <p>If you have questions about the assignment please post them in the LEA forum before the deadline. Don't wait until the last day to post questions.</p></div>

<div class="alert alert-block alert-danger">
<p><b>Put your answer in the PROVIDED CELLS only!</b></p>
<b>Any new cell is not visible during the grading.</b>
<p>There is one code and markdown cell available for each question, so that you can use one or both of them according to the type of the question.</p>
<p>Do not copy the metadata from one cell to another, it is unique to that cell only.</p></div>

In [1]:
'''
Group Work:
Enter the UID (i.e. student2s) of each team member into the variables. 
If you work alone please leave the second variable empty.
'''
member1 = 'dpadma2s'
member2 = 'jbandl2s'
member3 = 'smuthi2s'

In [2]:
'''
Put here the average solution time for the full exercise set in estimated MINUTES".
AverageSolutionTimeinMinutes should be > or < 4711
i.e.
AverageSolutionTimeinMinutes = 180
'''

AverageSolutionTimeinMinutes = 840

In [3]:
# This is a test cell, please ignore it!

 # Task 1 : Pose three questions (and ans.) to last lecture [1 point]
 The format of the question and answer should be [Q1,A1,Q2,A2,Q3,A3], where Q1 is the question and A1 is the answer.
Both Q and A should contain equal to or more than three words. Do not remove any markdown tag like $<$br$>$ in the answer cell.

Q1 = What are some situations that require estimation of homography?
<br>
A1 = Few of the situations are:
* Camera calibration
* 3D reconstruction
* Visual metrology
* Stereo vision
* Scene understanding
* Automatic Mosaicing
<br>
<br>
Q2 = Why line based approaches are better than point based approaches in homography estimation?
<br>
A2 = It is a conventional belief that line-based approaches perform better than point-based ones for homography estimation, as the linefitting is generally more noise resistant than point detection.
<br>
<br>
Q3 = When can two images be related by homography?
<br>
A3 = Two images can have relation through homography if and only if:
* Both images are viewing the same plane from a different prespective.
* Both images are taken from the same camera but from different angle.
* Camera should be rotated along its origin of projection and there should not be any translation.
* Homography relation is independent of scene structure. So, it do not depend on what cameras are capturing and still relationship can be established irrespective of what is seen in images.

In [4]:
# This is a test cell, please ignore it!

# Task 2: Read parts of chapter 4 (Estimation – 2D Projective Transformations) of MVG namely 4.1, 4.4., 4.6., 4.7., 4.8. [1 point]<br>Produce some sort of a summary

<img src="boat/2DProjectiveTransformation-Estimation.png"/>

# Task 3: Implement Algor 4.2 (normalized DLT) in python by HAND, don't use the in-build image tool boxes. [3 points]

In [5]:
import typing
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
import cv2

class Perform_DLT():
    
    def __init__(self):
        
        np.set_printoptions(suppress=True)
        
    def get_correspondence_point(self, points_file: str, img_x: np.ndarray, img_xprime: np.ndarray) -> np.ndarray:
        '''
        Parser to read the correspondence points in img1 and img2
        '''
        
        self.points_file = points_file
        self.img_x = img_x
        self.img_xprime = img_xprime

        f = open(self.points_file)

        image_points = dict()
        for n,line in enumerate(f.readlines()):

            line = line.replace('[','').replace(']','').strip()
            pointlist = line.split('=')[1].split(';')
            x = list()
            y = list()

            for point in pointlist:
                x.append(int(point.split(',')[0]))
                y.append(int(point.split(',')[1]))
            image_points[n] = np.vstack((x,y))

        return image_points[0], image_points[1]

    def normalize(self, x: np.ndarray) -> np.ndarray:
        
        '''
        Normalizes a given matrix, x by performing similarity transformation.
        Now the centroid of the new points has (0,0)^T and the average distance 
        from the origin is square.root(2)
        '''

        x_bar = np.asarray(x)
        m = np.mean(x,1)

        deno = list()

        for i in x.transpose():
            deno.append(list(i-m))
        deno = np.square(np.array(deno))
        total = 0
        for i in deno:
            total = sum(i)**(1/2.) + total

        s = 2**(1/2)/(total/len(deno))

        T = np.array([[s, 0, -s*m[0]], [0, s, -s*m[1]], [0, 0, 1]])

        x = np.vstack((x, np.ones((1,x.shape[1]))))

        x_bar = np.dot(T, x)
        
        return x_bar, T

    def DLT(self, x: np.ndarray, x_bar: np.ndarray) -> np.ndarray:
        '''
        Performs DLT on the x and x' provided
        '''

        # Initializations for A matrix
        a_12 = list()
        a_13 = list()
        a_21 = list()
        a_23 = list()
        result = list()
        
        # Forming A matrix as provided in Reference 1 and Hartley and zisserman multiple view geometry

        for i,j in zip(x.transpose(), x_bar.transpose()):

            a_12= np.array([list(-np.dot(j[2],i.transpose()))])
            a_13= np.array([list(np.dot(j[1],i.transpose()))])
            a_21= np.array([list(np.dot(j[2],i.transpose()))])
            a_23= np.array([list(-np.dot(j[0],i.transpose()))])

            r1 = np.hstack((np.hstack((np.zeros((1,3)),a_12)),a_13))
            r2 = np.hstack((np.hstack((a_21,np.zeros((1,3)))),a_23))

            a = np.vstack((r1,r2))   

            for k in a:
                result.append(list(k))

        A = np.array(result)
        
        
        # Ah = 0 ;Minimizing ||Ah|| such that ||h||=1
        # Solution by SVD
        
        U,s,Vt = np.linalg.svd(A)
        V=Vt.T
        h=V[:,-1] # Last column of V corresponding eigen vector to the smallest eigen value
        
        h = np.reshape(h,(3,3))

        return h
    
    def desired_H(self, T_bar, T_prime_bar, H_normalized):
        
        '''
        Used for finding H in unnormalized original co-ordinates from normalized H' by performing,
        H = inverse(T') H' T
        '''
        non_scaled_H = np.dot(np.linalg.inv(T_prime_bar),(np.dot(H_normalized, T_bar)))
        # Scaling H such that H(3,3) is 1.
        desired_H = non_scaled_H/non_scaled_H[2][2]
        
        return desired_H
    
    
img_path1 = 'boat/img1+points.png'
img_path2 = 'boat/img2+points.png'
img1 = image.imread(img_path1)
img2 = image.imread(img_path2)
points_file = 'boat/homography.txt'

homography = Perform_DLT()

# Get initial corresponding points
x, x_prime = homography.get_correspondence_point(points_file, img1, img2)   

# Normalize x to overcome the numerical calculation issues in DLT
x_bar, T_bar = homography.normalize(x)

# Normalize x' to overcome the numerical calculation issues in DLT
x_prime_bar, T_prime_bar = homography.normalize(x_prime)

# Find H normalized
H_normalized = homography.DLT(x_bar, x_prime_bar)

# Find H mapping to original image scales.
H = homography.desired_H(T_bar, T_prime_bar, H_normalized)

print('Desired Homography, H:\n', H)

Desired Homography, H:
 [[  0.87545097   0.23238266   3.4699619 ]
 [ -0.21413475   0.87925678 129.60815544]
 [  0.00001108   0.0000304    1.        ]]


Steps:
1. Read correspondence points in 2 images.
2. Normalize x and x' to overcome the numerical calculation issues in DLT.
3. Find H normalized (H') by performing DLT.
Ah = 0 ;Minimizing ||Ah|| such that ||h||=1
h - Last column of V corresponding eigen vector to the smallest eigen value
4. Find H mapping to original image scales.
Used for finding H in unnormalized original co-ordinates from normalized H' by performing,
H = inverse(T') H' T

Desired Homography, H:
 [[  0.87545097   0.23238266   3.4699619 ]
 [ -0.21413475   0.87925678 129.60815544]
 [  0.00001108   0.0000304    1.        ]]

# Task 4: Validate your results using the test images and by comparison TO the in build image library methods! Use the images found in the "./boat/" sub-dir (or any other you pick: needed: 2 images and two lists of corresponding point pair coordinates,pls provide MORE test cases)[1 point]<br>NB I am not 100% sure that the "boat" list is matching..)

In [6]:
import typing
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
import cv2

class Perform_DLT():
    
    def __init__(self):
        
        np.set_printoptions(suppress=True)
        
    def get_correspondence_point(self, points_file: str, img_x: np.ndarray, img_xprime: np.ndarray) -> np.ndarray:
        '''
        Parser to read the correspondence points in img1 and img2
        '''
        
        self.points_file = points_file
        self.img_x = img_x
        self.img_xprime = img_xprime

        f = open(self.points_file)

        image_points = dict()
        for n,line in enumerate(f.readlines()):

            line = line.replace('[','').replace(']','').strip()
            pointlist = line.split('=')[1].split(';')
            x = list()
            y = list()

            for point in pointlist:
                x.append(int(point.split(',')[0]))
                y.append(int(point.split(',')[1]))
            image_points[n] = np.vstack((x,y))

        return image_points[0], image_points[1]

    def normalize(self, x: np.ndarray) -> np.ndarray:
        
        '''
        Normalizes a given matrix, x by performing similarity transformation.
        Now the centroid of the new points has (0,0)^T and the average distance 
        from the origin is square.root(2)
        '''

        x_bar = np.asarray(x)
        m = np.mean(x,1)

        deno = list()

        for i in x.transpose():
            deno.append(list(i-m))
        deno = np.square(np.array(deno))
        total = 0
        for i in deno:
            total = sum(i)**(1/2.) + total

        s = 2**(1/2)/(total/len(deno))

        T = np.array([[s, 0, -s*m[0]], [0, s, -s*m[1]], [0, 0, 1]])

        x = np.vstack((x, np.ones((1,x.shape[1]))))

        x_bar = np.dot(T, x)
        
        return x_bar, T

    def DLT(self, x: np.ndarray, x_bar: np.ndarray) -> np.ndarray:
        '''
        Performs DLT on the x and x' provided
        '''

        # Initializations for A matrix
        a_12 = list()
        a_13 = list()
        a_21 = list()
        a_23 = list()
        result = list()
        
        # Forming A matrix as provided in Reference 1 and Hartley and zisserman multiple view geometry

        for i,j in zip(x.transpose(), x_bar.transpose()):

            a_12= np.array([list(-np.dot(j[2],i.transpose()))])
            a_13= np.array([list(np.dot(j[1],i.transpose()))])
            a_21= np.array([list(np.dot(j[2],i.transpose()))])
            a_23= np.array([list(-np.dot(j[0],i.transpose()))])

            r1 = np.hstack((np.hstack((np.zeros((1,3)),a_12)),a_13))
            r2 = np.hstack((np.hstack((a_21,np.zeros((1,3)))),a_23))

            a = np.vstack((r1,r2))   

            for k in a:
                result.append(list(k))

        A = np.array(result)
        
        
        # Ah = 0 ;Minimizing ||Ah|| such that ||h||=1
        # Solution by SVD
        
        U,s,Vt = np.linalg.svd(A)
        V=Vt.T
        h=V[:,-1] # Last column of V corresponding eigen vector to the smallest eigen value
        
        h = np.reshape(h,(3,3))

        return h
    
    def desired_H(self, T_bar, T_prime_bar, H_normalized):
        
        '''
        Used for finding H in unnormalized original co-ordinates from normalized H' by performing,
        H = inverse(T') H' T
        '''
        non_scaled_H = np.dot(np.linalg.inv(T_prime_bar),(np.dot(H_normalized, T_bar)))
        # Scaling H such that H(3,3) is 1.
        desired_H = non_scaled_H/non_scaled_H[2][2]
        
        return desired_H
    

def test_case1(img1_path: str, img2_path:str, points_file: str):
    '''
    TEST: Given 2 images and correspondence points,
    compares H from implemented normalized DLT and
    H from openCV findHomography function.
    
    Provides visual outputs illustrating the comparison.
    '''   
    
    print('***** Executing test case 1 *****\n')
    img1 = cv2.imread('boat/img1+points.png')
    img2 = cv2.imread('boat/img2+points.png')

    img2_observed = deepcopy(img2)
    img2_cv = deepcopy(img2)

    homography1 = Perform_DLT()
    x, x_prime = homography1.get_correspondence_point(points_file, img1, img2)

    homography1 = Perform_DLT()
    x, x_prime = homography1.get_correspondence_point(points_file, img1, img2)   
    x_bar, T_bar = homography1.normalize(x)
    x_prime_bar, T_prime_bar = homography1.normalize(x_prime)
    H_normalized = homography1.DLT(x_bar, x_prime_bar)
    H = homography1.desired_H(T_bar, T_prime_bar, H_normalized)
    print("Homography estimated using the implemented Normalized DLT:\n", H)

    H_cv, status = cv2.findHomography(x.transpose(), x_prime.transpose())
    print("Homography estimated using OPENCV:\n", H_cv)

    for i in zip(x[0], x[1]):
        cv2.circle(img1,(i),1,(0,0,255), 5)

    for i in zip(x_prime[0], x_prime[1]):
        cv2.circle(img2,(i),1,(0,0,255), 5) # RED
        cv2.circle(img2_observed, i, 1, (0,0,255),5) # RED
        cv2.circle(img2_cv, i, 1, (0,0,255),5) # RED

    x_p_observed = np.dot(H,np.vstack((x,np.ones((1,x.shape[1])))))
    x_p_observed = x_p_observed.astype(int)

    x_p_cv = np.dot(H_cv,np.vstack((x,np.ones((1,x.shape[1])))))
    x_p_cv = x_p_cv.astype(int)

    for i in zip(x_p_observed[0], x_p_observed[1]):
        cv2.circle(img2, i, 1, (255,0,0), 5) # BLUE
        cv2.circle(img2_observed, i, 1, (255,0,0),5) # BLUE

    for i in zip(x_p_cv[0], x_p_cv[1]):
        cv2.circle(img2, i, 1, (0,255,0), 5) # GREEN
        cv2.circle(img2_cv, i, 1, (0,255,0),5) # GREEN
        
    print('\nGROUND TRUTH COMPARISON FOR IMG1 -> IMG2 TRANSFORMATION:')
    
    label1 = 'Red- Ground truth, Blue and Green - Using H from implemented DLT and OpenCV on img1 points'
    cv2.putText(img2,label1, (20,20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2, cv2.LINE_AA )
    cv2.imwrite('boat/result_all.png', img2)
    print('\nComparison of ground truth (red), transformed points from implemented Normalized DLT (blue) and opencv (green) is saved in boat/result_all.png')
    
    label2 = 'Red- Ground truth, Blue- Using H from implemented DLT on img1 points'
    cv2.putText(img2_observed,label2, (20,20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2, cv2.LINE_AA )
    cv2.imwrite('boat/result_observed.png', img2_observed)
    print('\nComparison of ground truth (red) and transformed points from implemented Normalized DLT (blue) is saved in boat/result_observed.png')
    
    label3 = 'Red- Ground truth, Green- Using H from OpenCV on img1 points'
    cv2.putText(img2_cv,label3, (20,20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2, cv2.LINE_AA )
    cv2.imwrite('boat/result_cv.png', img2_cv)
    

    print('\nComparison of ground truth (red) and transformed points from implemented Normalized DLT (blue) is saved in boat/result_cv.png')  
    
    print('\n***** Test case 1 passed *****\n')
    
def test_case2(img1_path: str, img2_path:str, points_file: str):
    
    '''
    TEST: Given 2 set of correspondence points in 2 images, 
    compares H from implemented normalized DLT and
    H from openCV findHomography function.
    
    Provides error value with each term of H matrix.
    
    The test passes if the error in all terms is below a tolerance say 0.05.
    '''
    
    print('\n***** Executing test case 2 *****\n')
    homography1 = Perform_DLT()
    img1 = cv2.imread('boat/img1+points.png')
    img2 = cv2.imread('boat/img2+points.png')
    x, x_prime = homography1.get_correspondence_point(points_file, img1, img2)   
    x_bar, T_bar = homography1.normalize(x)
    x_prime_bar, T_prime_bar = homography1.normalize(x_prime)
    H_normalized = homography1.DLT(x_bar, x_prime_bar)
    H = homography1.desired_H(T_bar, T_prime_bar, H_normalized)
    print("Homography estimated using the implemented Normalized DLT:\n", H)

    H_cv, status = cv2.findHomography(x.transpose(), x_prime.transpose())
    print("Homography estimated using OPENCV:\n", H_cv)
    
    error = ((H-H_cv)/H_cv)
    print('Error in percentage (for each cases):\n',error)
    
    difference = H-H_cv
    
    assert np.all(difference < 0.05),'Test case 2 failed'
    print('\n***** Test case 2 passed *****\n')

    
def test_case3():
    
    '''Test case for DLT only (without normalization)
    This is a DLT simple example case taken from 
    http://read.pudn.com/downloads75/ebook/273942/estimation-4.pdf
    
    In this test case, we check whether our implementation results
    matches with the analytical solution provided in the reference.
    '''
    print('***** Executing test case 3 *****\n')
    x = np.array([[500,500,600,700,700],[500,700,600,500,700],[1,1,1,1,1]])
    x_prime = np.array([[501,500,600,700,700],[500,700,600,500,700],[1,1,1,1,1]])
    H_actual_answer = np.array([[0.970, -0.018, 16.030],[-0.006,0.963, 12.741],[0.000, 0.000, 1.000]])

    test_case = Perform_DLT()
    non_scaled_H = test_case.DLT(x,x_prime)
    H_test_case = non_scaled_H/non_scaled_H[2][2]
    H_test_case = np.round(H_test_case, 3)
    
    print('Actual H value for test case:\n',H_actual_answer)
    print('Observed H value for test case:\n', H_test_case)
    
    assert np.allclose(np.squeeze(H_actual_answer), np.squeeze(H_test_case)),'Test case 3 failed'
    print('\n***** Test case 3 passed *****\n')

img_path1 = 'boat/img1+points.png'
img_path2 = 'boat/img2+points.png' 
points_file = 'boat/homography.txt'
test_case1(img_path1, img_path2, points_file)
test_case2(img_path1, img_path2, points_file)
test_case3()



***** Executing test case 1 *****

Homography estimated using the implemented Normalized DLT:
 [[  0.87545097   0.23238266   3.4699619 ]
 [ -0.21413475   0.87925678 129.60815544]
 [  0.00001108   0.0000304    1.        ]]
Homography estimated using OPENCV:
 [[  0.87562125   0.23248716   3.42174345]
 [ -0.21411874   0.87937677 129.60221821]
 [  0.00001124   0.00003059   1.        ]]

GROUND TRUTH COMPARISON FOR IMG1 -> IMG2 TRANSFORMATION:

Comparison of ground truth (red), transformed points from implemented Normalized DLT (blue) and opencv (green) is saved in boat/result_all.png

Comparison of ground truth (red) and transformed points from implemented Normalized DLT (blue) is saved in boat/result_observed.png

Comparison of ground truth (red) and transformed points from implemented Normalized DLT (blue) is saved in boat/result_cv.png

***** Test case 1 passed *****


***** Executing test case 2 *****

Homography estimated using the implemented Normalized DLT:
 [[  0.87545097   0.23238

From Test case 1, it is clear that the two set of corresponding points of image 1 and image 2 matches, find the uploaded files in the name boat/result_all.png, boat/result_observed.png, boat/result_cv.png. However, the visual and empirical results portray there is a tolerable error in mapping the points by homography between opencv and the implemented method with the ground truth provided. Test case 2 also illustrate this effect in terms of relative error with each element of the matrix.

Test case 3 from reference 2 provided a solution for DLT from a set of 5 correspondence points. This test case checks whether the implementation is able to agree with the solution provided in the reference (an additional test case to check the empirical correctness of the DLT implemented). 

References:

1. Rong Zhang, 'Automatic Computation of a Homography byRANSAC Algorithm', Purdue University, Accessed on: 25.04.2020, URL: https://engineering.purdue.edu/kak/courses-i-teach/ECE661.08/solution/hw4_s1.pdf.

2. Estimation notes, Accessed on: 25.04.2020, URL: http://read.pudn.com/downloads75/ebook/273942/estimation-4.pdf.